In [1]:
!rm -rf ./logs/ 
%run Model.ipynb
%run dataset_loader.ipynb
import tensorflow as tf
#from dataset_loader import load_dataset

tf.enable_eager_execution()
INPUT_WIDTH = 256
INPUT_HEIGHT = 256
INPUT_CHANNELS = 3

NUM_CLASSES = 5

LEARNING_RATE = 0.001   # Original value: 0.01
MOMENTUM = 0.9
KEEP_PROB = 1

EPOCHS = 1
BATCH_SIZE = 128

train_sub_dir = '/data/image_format/*/*'
val_sub_dir = '/data/image_format_small_dataset_val/*/*'
test_sub_dir = '/data/image_format_small_dataset_test/*/*'

frequency_train_sub_dir = '/data/cogisen_frequency_data/image_format_small_dataset_cogisen_updated/*/*/*.csv'
frequency_val_sub_dir = '/data/image_format_small_dataset_val_cogisen_updated/csv/*/*'
frequency_test_sub_dir = '/data/image_format_small_dataset_test_cogisen_updated/csv/*/*'

home_dir = os.path.dirname(os.path.realpath('__file__'))

train_dir = home_dir + train_sub_dir
val_dir = home_dir + val_sub_dir
test_dir = home_dir + test_sub_dir


frequency_train_dir = home_dir + frequency_train_sub_dir
frequency_val_dir = home_dir + frequency_val_sub_dir
frequency_test_dir = home_dir + frequency_test_sub_dir



training_dataset= load_dataset(image_dir= train_dir,frequency_dir=frequency_train_dir,minibatch_size=BATCH_SIZE)
# val_dataset = load_dataset(data_dir= val_dir,minibatch_size=BATCH_SIZE)
# test_dataset = load_dataset(data_dir = test_dir,minibatch_size=BATCH_SIZE)

experiment_no = 2
#logdir = './log/alexnet/FULL_DATA'+str(experiment_no)
logdir = './log/rubbish'





/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shubham/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.29 µs
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [3]:
csv_dataset = training_dataset[1]


In [4]:
image_dataset = training_dataset[0]

In [5]:
csv_dataset = csv_dataset.unbatch()

AttributeError: 'DatasetV1Adapter' object has no attribute 'unbatch'

In [ ]:
csv_dataset = csv_dataset.map(lambda x: list(x.unbatch.values()))

In [ ]:

for element in csv_dataset:
    print(element)
    break
    

In [ ]:
np.set_printoptions(precision=3, suppress=True)

In [ ]:
def show_batch(dataset):
  for batch in dataset.take(1):
    for  value in batch:
      print(" {}".format(value.numpy()))

In [ ]:
tf.executing_eagerly()

In [ ]:
show_batch(csv_dataset)

In [ ]:
d = tf.data.Dataset.concatenate(image_dataset,csv_dataset)

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:

handle = tf.placeholder(tf.string, shape=[])


training_iterator = tf.data.Iterator.from_structure(training_dataset.output_types,training_dataset.output_shapes)
training_init_op = training_iterator.make_initializer(training_dataset)




In [ ]:
csv_iterator = tf.data.Iterator.from_structure(csv_dataset.output_types,csv_dataset.output_shapes)
csv_init_op = training_iterator.make_initializer(csv_dataset)



In [ ]:
iterator = tf.data.Iterator.from_string_handle(
    handle, training_iterator.output_types,training_dataset.output_shapes)

next_training_element = iterator.get_next()

In [ ]:
csv_next_element = csv_iterator.get_next()

In [ ]:
with tf.device('/gpu:0'):
    alexnet = AlexNet(next_training_element,input_height=INPUT_HEIGHT,input_width=INPUT_WIDTH, num_classes=NUM_CLASSES)

In [ ]:
with tf.device('/gpu:0'):

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
        print('Training dataset...')
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        file_writer = tf.summary.FileWriter(logdir=logdir, graph=sess.graph)

        training_string_handle =  sess.run(training_iterator.string_handle())

        summary_operation = tf.summary.merge_all()

        for epoch in range(EPOCHS):

            sess.run(training_init_op,feed_dict={handle:training_string_handle})
            _, accuracy,summary,Y,final_logits,csv = sess.run([alexnet.training_operation,alexnet.accuracy_operation,summary_operation,alexnet.Y,alexnet.final_logits,csv_dataset],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob,handle:training_string_handle})


            file_writer.add_summary(summary, epoch)
            print("accuracy for epoch {0}  :  {1}".format(epoch,accuracy))

            #print("Y: ",Y)
            #print("logits: ",final_logits)
            print("csv: ",csv)
            

In [ ]:

val_iterator = tf.data.Iterator.from_structure(val_dataset.output_types,val_dataset.output_shapes)
val_init_op = val_iterator.make_initializer(val_dataset)



In [ ]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    print('validation dataset...')
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(val_init_op)
    val_string_handle =  sess.run(val_iterator.string_handle())
    val_accuracy = sess.run([alexnet.accuracy_operation],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob, 
                                                                     handle:val_string_handle})
    

    print("accuracy for validation set :  {0}".format(val_accuracy))



In [ ]:

test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,test_dataset.output_shapes)
test_init_op = test_iterator.make_initializer(test_dataset)



In [ ]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    print('Testing dataset...')
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    sess.run(test_init_op)
    test_string_handle =  sess.run(test_iterator.string_handle())
    test_accuracy = sess.run([alexnet.accuracy_operation],feed_dict={alexnet.dropout_keep_prob: alexnet.keep_prob, 
                                                                     handle:test_string_handle})
    

    print("accuracy for test set :  {0}".format(test_accuracy))

